In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.set_option('display.float_format', lambda x: '%.5f' % x)

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
target = 'duration'

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1716819305993, experiment_id='1', last_update_time=1716819305993, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## Full base

In [3]:
URL_DATASET_JANUARY = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
URL_DATASET_FEBRUARY = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

df = pd.read_parquet(URL_DATASET_JANUARY)
print(f'Number of columns: {len(df.columns)}')

df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

print(f'Standard deviation: {df.duration.std()}')

Number of columns: 19
Standard deviation: 42.59435124195458


## Remove outliers

In [4]:
def read_dataframe(url)->pd.DataFrame:
    df = pd.read_parquet(url)

    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df[categorical] = df[categorical].astype(str)
    
    return df

df_train = read_dataframe(URL_DATASET_JANUARY)

print(f'Fraction of the records left : {df_train.shape[0] / df.shape[0] * 100}')

Fraction of the records left : 98.1220282212598


In [5]:
# Training pipeline
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df_train[target].values

## Validation

In [7]:
# Validation
df_validation = read_dataframe(URL_DATASET_FEBRUARY)

validation_dicts = df_validation[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(validation_dicts)
y_val = df_validation[target].values


## Save model

In [ ]:
# with mlflow.start_run():
#     mlflow.set_tag('developer', 'Quentin')
#     mlflow.log_param('train-data', 'yellow_tripdata_2023-01.parquet')
#     mlflow.log_param('validation-data', 'yellow_tripdata_2023-02.parquet')
    
#     alpha = 0.001
#     mlflow.log_param('alpha', alpha)

#     lr = Lasso(alpha)
#     lr.fit(X_train, y_train)

#     y_pred = lr.predict(X_val)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
#     mlflow.log_metric('rmse', rmse)


In [8]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [9]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [30]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('developer', 'Quentin')
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)

    return {'loss': rmse, 'status': STATUS_OK}


In [31]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:8.70287               
[1]	validation-rmse:7.65152               
[2]	validation-rmse:6.84908               
[3]	validation-rmse:6.25036               
[4]	validation-rmse:5.81022               
[5]	validation-rmse:5.48701               
[6]	validation-rmse:5.25501               
[7]	validation-rmse:5.08965               
[8]	validation-rmse:4.96955               
[9]	validation-rmse:4.88396               
[10]	validation-rmse:4.82203              
[11]	validation-rmse:4.77654              
[12]	validation-rmse:4.74181              
[13]	validation-rmse:4.71605              
[14]	validation-rmse:4.69587              
[15]	validation-rmse:4.68180              
[16]	validation-rmse:4.67003              
[17]	validation-rmse:4.66132              
[18]	validation-rmse:4.65319              
[19]	validation-rmse:4.64527              
[20]	validation-rmse:4.64053              
[21]	validation-rmse:4.63619              
[22]	validation-rmse:4.63120              
[23]	valida

KeyboardInterrupt: 

In [29]:
mlflow.end_run()